In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a forcasting problem, where the goal is to predict the number of cycles an engine will last before it fails.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [3]:
!pip install pytorch-lightning
!pip install colorlog
!pip install bayesian-optimization
!pip install pandas
!pip install plotly
!pip install matplotlib
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
# third-party libraries
import pandas as pd
import numpy as np
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from bayes_opt import BayesianOptimization

import time

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# source code
from src.data_loading import load_data, load_config
from src.data_splitting import train_val_split_by_group
from src.nn_utils import create_sliding_window, create_sliding_window_test
from src.rolling_window_creator import calculate_RUL
from src.data_processing import apply_padding_on_train_data_and_test_data, drop_samples_with_clipped_values
from src.nn_util.nn_models.ligthning.cnnModel1 import CNNModel1 as CNNModel
from src.nn_util.datamodule.lightning.turbofanDatamodule import TurbofanDatamodule
from src.data_cleaning import clean_data

In [5]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [6]:
np.random.seed(42)

# Paths

In [7]:
# Make sure to execute this cell only once for one kernel session, before running any other cell below.
os.chdir("../../") # set working directory to root of project
os.getcwd() # check current working directory

'C:\\Users\\Johannes\\PycharmProjects\\damage-propagation-modeling'

In [8]:
PATH_TO_CONFIG = "configs/config.yaml"

# Load Config + Data

In [9]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [10]:
train_data, test_data, test_RUL_data = load_data(config_path=PATH_TO_CONFIG, dataset_num=1)

2024-05-31 22:44:57 [src.data_loading:43] [INFO] >>>> Loading data set 1...
2024-05-31 22:44:57 [src.data_loading:72] [INFO] >>>> Loaded raw data for dataset 1.
2024-05-31 22:44:57 [src.data_loading:73] [INFO] >>>> Train Data: (20631, 26)
2024-05-31 22:44:57 [src.data_loading:74] [INFO] >>>> Test Data: (13096, 26)
2024-05-31 22:44:57 [src.data_loading:75] [INFO] >>>> Test RUL Data: (100, 1)


# 📍 << Subtask X: TOPIC >>

[TEMPLATE]

Findings:
* Interpretation of plots
* or other key take aways from previous code

TODO: Write down explanation of pipeline!

In [11]:
# some hyperparameters
time_column = 'Cycle'
group_column = 'UnitNumber'

window_size = 30
clip_value = 125
apply_data_cleaning = True

# Apply scaler. The order in the list represents the order in which they are applied
std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
robust_scaler = RobustScaler()
scaler = [std_scaler, minmax_scaler, robust_scaler]

# The model_type can be 'CNN_2D' or 'CNN_1D' and defines what type of convolutional layers has been used.
model_type = 'CNN_1D' 

# Setting the seed
pl.seed_everything(21)

Seed set to 21


21

In [12]:
if apply_data_cleaning:
    train_data, test_data = clean_data(train_data, test_data, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.0, contamination=0.05)
    
# Add column RUL to train_data
train_data = calculate_RUL(train_data, time_column, group_column, clip_value)

train_data, test_data = apply_padding_on_train_data_and_test_data(train_data=train_data, test_data=test_data, window_size=window_size)

X_test, _ = create_sliding_window_test(test_data, column_RUL=False, drop_columns=['UnitNumber'])
y_test = test_RUL_data.values

train, val = train_val_split_by_group(train_data, test_size=0.1, random_state=12)

X_train, y_train = create_sliding_window(train, window_size=window_size)  #, drop_columns=['UnitNumber', 'Cycle, 'RUL'])
X_val, y_val = create_sliding_window(val, window_size=window_size)  #, drop_columns=['UnitNumber', 'Cycle', 'RUL'])

X_train, y_train = drop_samples_with_clipped_values(X_train, y_train, clip_value)
X_val, y_val = drop_samples_with_clipped_values(X_val, y_val, clip_value)

2024-05-31 22:44:58 [src.data_cleaning:134] [INFO] >>>> Cleaning train and test data...
2024-05-31 22:44:58 [src.data_cleaning:136] [INFO] >>>> Formatting column types...
2024-05-31 22:44:58 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []
2024-05-31 22:44:58 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []
2024-05-31 22:44:58 [src.data_cleaning:141] [INFO] >>>> Handling duplicates...
2024-05-31 22:44:58 [src.data_cleaning:146] [INFO] >>>> Removing outliers...
2024-05-31 22:44:58 [src.outlier_detection:150] [DEBUG] >>>> Removing outliers using method: None ...
2024-05-31 22:44:58 [src.outlier_detection:162] [INFO] >>>> No outlier detection method specified. Skipping outlier detection.
2024-05-31 22:44:58 [src.outlier_detection:150] [DEBUG] >>>> Removing outliers using method: None ...
2024-05-31 22:44:58 [src.outlier_detection:162] [INFO] >>>> No outlier detection method specified. Skipping outlier detection.
2024-05-31 22:44:58 [src.data_cleaning:

Scale the data

In [13]:
X_train_shape = X_train.shape
X_val_shape = X_val.shape
X_test_shape = X_test.shape

# Note: Do not normalize the cycle value! That is why we start with one
for single_scaler in scaler:
    for i in range(1, X_train.shape[-1]):
        X_train[:, :, i] = single_scaler.fit_transform(X_train[:, :, i])
        X_val[:, :, i] = single_scaler.transform(X_val[:, :, i])
        X_test[:, :, i] = single_scaler.transform(X_test[:, :, i])

Change data types of arrays to float32 and swap axes if necessary:

In [14]:
model_type = 'CNN_1D'

print(X_train.shape)
if model_type == 'CNN_1D':
    X_train = np.swapaxes(X_train, 1, 2)
elif model_type == 'CNN_2D':
    X_train = np.swapaxes(X_train, 1, 2)
    X_train = X_train[:, np.newaxis, :, :]
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
print(X_train.shape)

print(X_val.shape)
if model_type == 'CNN_1D':
    X_val = np.swapaxes(X_val, 1, 2)
elif model_type == 'CNN_2D':
    X_val = np.swapaxes(X_val, 1, 2)
    X_val = X_val[:, np.newaxis, :, :]
X_val = np.array(X_val, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)
print(X_val.shape)

print(X_test.shape)
if model_type == 'CNN_1D':
    X_test = np.swapaxes(X_test, 1, 2)
elif model_type == 'CNN_2D':
    X_test = np.swapaxes(X_test, 1, 2)
    X_test = X_test[:, np.newaxis, :, :]
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)
print(X_test.shape)

(11250, 30, 18)
(11250, 18, 30)
(1260, 30, 18)
(1260, 18, 30)
(100, 30, 18)
(100, 18, 30)


## Hyperparameter Search 

In [ ]:
from sklearn.metrics import root_mean_squared_error
import torch 
# Setting the seed
pl.seed_everything(21)

def hyperparameter_function_cnn(learning_rate_init, batch_size, dropout, beta_1, beta_2):
    """ Function for hyperparameter optimization
    """    
    # Select hyperparameters of trainer!
    checkpoint_callback = ModelCheckpoint(monitor="val_loss")
    trainer = Trainer(min_epochs=1, max_epochs=150, callbacks=[checkpoint_callback], deterministic=True)
    datamodule = TurbofanDatamodule(batch_size=int(batch_size), random_state=15)
    datamodule.set_train_dataset(X_train, y_train)
    datamodule.set_val_dataset(X_val, y_val)
    datamodule.set_predict_dataset(X_test)
    datamodule.set_test_dataset(X_test, y_test)
    model = CNNModel(lr=learning_rate_init, window_size=window_size, features=18, dropout_rate=dropout, beta_1=beta_1, beta_2=beta_2)
    trainer.fit(model, datamodule=datamodule)
    pred = trainer.validate(model, datamodule=datamodule, ckpt_path='best')
    return - pred[0]['val_loss']
    

# Bounded region of parameter space
pbounds = {'learning_rate_init': (0.00001, 0.1), 'batch_size': (10, 200), 'dropout': (0.01, 0.5), 'beta_1': (0.8, 0.999), 'beta_2': (0.9, 0.999999)}

optimizer = BayesianOptimization(
    f=hyperparameter_function_cnn,
    pbounds=pbounds,
    random_state=14,
)
optimizer.maximize(
    init_points=20,
    n_iter=30,
)

In [16]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -380.0711669921875, 'params': {'batch_size': 107.64923531681114, 'beta_1': 0.9538598453637801, 'beta_2': 0.9870418981447955, 'dropout': 0.013943004779610777, 'learning_rate_init': 0.03098049519126536}}
Iteration 1: 
	{'target': -623.3966674804688, 'params': {'batch_size': 191.94471047951927, 'beta_1': 0.9021102257413096, 'beta_2': 0.931828124202785, 'dropout': 0.27420796933856756, 'learning_rate_init': 0.022133281693351714}}
Iteration 2: 
	{'target': -1321.9324951171875, 'params': {'batch_size': 163.2314580056113, 'beta_1': 0.8681086704973222, 'beta_2': 0.9538883460178654, 'dropout': 0.012878154920018581, 'learning_rate_init': 0.06731851629327074}}
Iteration 3: 
	{'target': -3635.07421875, 'params': {'batch_size': 49.90460981396025, 'beta_1': 0.9855789609828637, 'beta_2': 0.9374241007033998, 'dropout': 0.37868527182211376, 'learning_rate_init': 0.07631626879453375}}
Iteration 4: 
	{'target': -1321.942626953125, 'params': {'batch_size': 175.39489650163253, 'bet

In [17]:
threshold = 500
under_threshold_indices = np.arange(len(optimizer.res))[np.array([- optimizer.res[i]["target"] for i in range(len(optimizer.res))]) < threshold]
under_threshold = [optimizer.res[i] for i in under_threshold_indices]

In [18]:
under_threshold

[{'target': -380.0711669921875,
  'params': {'batch_size': 107.64923531681114,
   'beta_1': 0.9538598453637801,
   'beta_2': 0.9870418981447955,
   'dropout': 0.013943004779610777,
   'learning_rate_init': 0.03098049519126536}},
 {'target': -388.8275451660156,
  'params': {'batch_size': 39.080953225229784,
   'beta_1': 0.8977959224520511,
   'beta_2': 0.9010728278918141,
   'dropout': 0.3270620437930545,
   'learning_rate_init': 0.007238704901270764}}]

In [19]:
max_params = optimizer.max['params']
print(optimizer.max)

{'target': -380.0711669921875, 'params': {'batch_size': 107.64923531681114, 'beta_1': 0.9538598453637801, 'beta_2': 0.9870418981447955, 'dropout': 0.013943004779610777, 'learning_rate_init': 0.03098049519126536}}


In [ ]:
def hyperparameter_function_cnn(learning_rate_init, batch_size, dropout, beta_1, beta_2):
    """ Function for hyperparameter optimization
    """    
    # Select hyperparameters of trainer!
    checkpoint_callback = ModelCheckpoint(monitor="val_loss")
    trainer = Trainer(min_epochs=1, max_epochs=150, callbacks=[checkpoint_callback], deterministic=True)
    datamodule = TurbofanDatamodule(batch_size=int(batch_size), random_state=15)
    datamodule.set_train_dataset(X_train, y_train)
    datamodule.set_val_dataset(X_val, y_val)
    datamodule.set_predict_dataset(X_test)
    datamodule.set_test_dataset(X_test, y_test)
    model = CNNModel(lr=learning_rate_init, window_size=window_size, features=18, dropout_rate=dropout, beta_1=beta_1, beta_2=beta_2)
    trainer.fit(model, datamodule=datamodule)
    pred = trainer.validate(model, datamodule=datamodule, ckpt_path='best')
    return - pred[0]['val_loss']
    

# Bounded region of parameter space
pbounds = {'learning_rate_init': (0.01, 0.1), 'batch_size': (100, 115), 'dropout': (0.005, 0.1), 'beta_1': (0.9, 0.99), 'beta_2': (0.95, 0.99)}

optimizer = BayesianOptimization(
    f=hyperparameter_function_cnn,
    pbounds=pbounds,
    random_state=17,
    allow_duplicate_points=True,
)
optimizer.maximize(
    init_points=20,
    n_iter=30,
)

In [17]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -1321.940185546875, 'params': {'batch_size': 104.41997504030664, 'beta_1': 0.9477528080044765, 'beta_2': 0.9576608314778997, 'dropout': 0.01145053402817268, 'learning_rate_init': 0.0808286913999922}}
Iteration 1: 
	{'target': -545.5281982421875, 'params': {'batch_size': 109.84500282663782, 'beta_1': 0.9573768806439272, 'beta_2': 0.9730241157501214, 'dropout': 0.008710977037942316, 'learning_rate_init': 0.04220322440351941}}
Iteration 2: 
	{'target': -208.14474487304688, 'params': {'batch_size': 114.18524780227611, 'beta_1': 0.9054040212283847, 'beta_2': 0.9845616841418484, 'dropout': 0.08834259998406288, 'learning_rate_init': 0.014607429905840833}}
Iteration 3: 
	{'target': -398.7694091796875, 'params': {'batch_size': 109.78627923198482, 'beta_1': 0.9496576231771388, 'beta_2': 0.973900530129562, 'dropout': 0.0509352193094502, 'learning_rate_init': 0.03546893447712921}}
Iteration 4: 
	{'target': -1321.97802734375, 'params': {'batch_size': 104.46588577452657, 'b

In [19]:
max_params = optimizer.max['params']
print(optimizer.max)

{'target': -136.14785766601562, 'params': {'batch_size': 114.84809532072403, 'beta_1': 0.9586517323123119, 'beta_2': 0.9558431375026947, 'dropout': 0.021025382021542985, 'learning_rate_init': 0.01}}
